In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB as GB
from keras.datasets import mnist
from sklearn.metrics import classification_report as CR
from sklearn.metrics import confusion_matrix as CF
from sklearn.metrics import f1_score
from math import log
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

Load data and split it:

In [ ]:
MNIIST=mnist.load_data()
#x,y = MNIIST["data"], MNIIST["target"]  ?????
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.2)
print(x_train.shape,x_test.shape)
re_x_train=x_train.reshape(60000,784)
re_x_test=x_test.reshape(10000,784)
print(re_x_train.shape,re_x_test.shape)

(60000, 28, 28) (10000, 28, 28)
(60000, 784) (10000, 784)


In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
x = mnist.data
y = mnist.target
X_train,X_test,Y_train,Y_test = train_test_split(x, y, test_size = 0.2)
print(x_train.shape,x_test.shape)
Xtrain = X_train.reshape(60000,784)
Xtest = X_test.reshape(10000,784)

Naive Bayes classifier:

In [ ]:
def caclculate_mean(x,y):
    #(rows,cols)=x.shape
    mean = np.zeros((len(np.unique(y)),  x.shape[1]))
    for i,j in enumerate(np.unique(y)):
         #row=axis(0)
         mean[i]=(x[y==j]).mean(axis=0)
    return mean

In [ ]:
def caclculate_variance(x,y):
    #(rows,cols)=x.shape
    Variance = np.zeros((len(np.unique(y)),  x.shape[1]))
    for i,j in enumerate(np.unique(y)):
         #row=(axis=0)
         Variance[i]=(x[y==j]).var(axis=0)
    return Variance
#caclculate_variance(X_train,Y_train)

In [ ]:
def caclculate_Probability_of_class(x,y):
    #(rows,cols)=x.shape
    Probability_of_class=np.zeros((len(np.unique(y))))
    for i,j in enumerate(np.unique(y)):
         #row=(axis=0)
         Probability_of_class[i]= ((x[y==j]).shape[0])/x.shape[0]
    return Probability_of_class


In [ ]:
def calculate_Liklihood(mean,Variance,x):
     Numerator = np.exp(-((x-mean)**2) / (2*Variance))
     Denominator = np.sqrt(2 *np.pi*Variance)
     liklihood=Numerator/Denominator
     return liklihood

In [ ]:
def predict(X, mean, Variance, Probability_of_class, classes):
    posteriors = []
    for x in X:
        likelihood = [calculate_Liklihood(mean[i], Variance[i], x) for i in range(len(classes))]
        log_likelihood = [np.sum(np.nan_to_num(np.log(z))) for z in likelihood]
        log_Probability_of_class = [np.log(Probability_of_class[i]) for i in range(len(classes))]
        posteriors_i = [log_likelihood[i] + log_Probability_of_class[i] for i in range(len(classes))]
        posteriors.append(posteriors_i)
    return [classes[np.argmax(posterior_i)] for posterior_i in posteriors]

In [ ]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])
mean=caclculate_mean(x_train, y_train)
Variance=caclculate_variance(x_train, y_train)
Probability_of_class=caclculate_Probability_of_class(x_train, y_train)
classes = np.unique(y_train)
predictions = predict(x_test, mean, Variance, Probability_of_class, classes)
print("classification_report",CR(y_test, predictions))
print("f1_score",f1_score(y_test, predictions, average=None))
print("confusion_matrix ",CF(y_test, predictions))
accuracy = (predictions == y_test).mean()
print("Accuracyof the model:", (accuracy*100),"%")


<ipython-input-5-1e1fa44a705b>:2: RuntimeWarning: divide by zero encountered in true_divide
  Numerator = np.exp(-((x-mean)**2) / (2*Variance))
<ipython-input-5-1e1fa44a705b>:2: RuntimeWarning: invalid value encountered in true_divide
  Numerator = np.exp(-((x-mean)**2) / (2*Variance))
<ipython-input-5-1e1fa44a705b>:4: RuntimeWarning: invalid value encountered in true_divide
  liklihood=Numerator/Denominator
<ipython-input-26-bc02b32ead22>:5: RuntimeWarning: divide by zero encountered in log
  log_likelihood = [np.sum(np.nan_to_num(np.log(z))) for z in likelihood]
/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


classification_report               precision    recall  f1-score   support

           0       0.83      0.90      0.87       980
           1       0.81      0.96      0.88      1135
           2       0.90      0.45      0.60      1032
           3       0.79      0.64      0.70      1010
           4       0.87      0.35      0.50       982
           5       0.69      0.15      0.24       892
           6       0.72      0.92      0.81       958
           7       0.88      0.42      0.57      1028
           8       0.38      0.68      0.49       974
           9       0.42      0.92      0.58      1009

    accuracy                           0.65     10000
   macro avg       0.73      0.64      0.62     10000
weighted avg       0.73      0.65      0.63     10000

f1_score [0.86650367 0.87710843 0.60116354 0.70394737 0.50361795 0.24236818
 0.80730594 0.57124097 0.49260355 0.57809583]
confusion_matrix  [[ 886    1    3    4    3    8   29    2   30   14]
 [   0 1092    2    3    0